Hay que normalizar los datos numéricos y transformar los datos categóricos para poder utilizarlos en los modelos. Utilizar *Min-Max Scaling* escala los datos en un rango [0,1], sirve para cualquier distribución, y se usa mucho en algunos modelos de aprendizaje automático. 

In [27]:
import pandas as pd
import numpy as np
import os


#os.chdir("C:/Users/cuent/En-Peu/notebooks/")
print(os.getcwd())

C:\Users\cuent\En-Peu\notebooks


In [28]:
# Cambiar al directorio padre
os.chdir("..")
parent_directory = os.getcwd()

# os.path.join() maneja correctamente las barras de ruta para diferentes SO (Windows, Linux, macOS)
file_path = os.path.join(parent_directory, 'data', 'data_stored.csv')

# Cargar el archivo CSV
data = pd.read_csv(file_path)


# Verificar la estructura del DataFrame (equivalente a str(data) en R)
print(data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64345 entries, 0 to 64344
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   POINT_X  64345 non-null  float64
 1   POINT_Y  64345 non-null  float64
 2   PLA      64345 non-null  float64
 3   A        64345 non-null  float64
 4   AC       64345 non-null  float64
 5   CIB      64345 non-null  float64
 6   CICCB    64345 non-null  float64
 7   CUS      64345 non-null  int64  
 8   DBB      64345 non-null  float64
 9   DOB      64345 non-null  float64
 10  OC       64345 non-null  int64  
 11  P        64345 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 5.9 MB
None


In [29]:
# Podemos ver que CUS y OC no se están contando como categóricas. Hay que transformarlas
data['CUS'] = data['CUS'].astype('category')
data['OC'] = data['OC'].astype('category')

# --- Identificar variables numéricas excluyendo las coordenadas ---
# Lista de columnas que se consideran coordenadas espaciales
coordenadas = ['POINT_X', 'POINT_Y']

# Seleccionar todas las columnas numéricas
numericas = data.select_dtypes(include=np.number).columns.tolist()

# Filtrar las columnas numéricas para excluir las coordenadas
# Usamos un conjunto para una búsqueda eficiente de coordenadas
cols_to_scale = [col for col in numericas if col not in set(coordenadas)]

print(f"\nColumnas a escalar (numéricas excluyendo coordenadas): {cols_to_scale}")



Columnas a escalar (numéricas excluyendo coordenadas): ['PLA', 'A', 'AC', 'CIB', 'CICCB', 'DBB', 'DOB', 'P']


In [30]:
# --- 1. Calcular Min y Max para las variables a escalar ---
# Almacenamos estos valores en un DataFrame para su fácil guardado
min_max_values = pd.DataFrame({
    'min': data[cols_to_scale].min(),
    'max': data[cols_to_scale].max()
})

print("\nValores Min y Max calculados:")
print(min_max_values)

min_max_values.to_csv(os.path.join(parent_directory, 'data', "min_max_values.csv"), index=True) # index=True para guardar los nombres de las variables


Valores Min y Max calculados:
            min          max
PLA    0.000000     9.280740
A      0.000000    29.951000
AC     3.000000   225.050000
CIB    0.098358     0.949654
CICCB  0.100001     0.999806
DBB    0.000000  1379.209961
DOB    0.000000  1407.369995
P      0.000000    28.388219


In [31]:
# --- 3. Aplicar Min-Max Scaling a las variables seleccionadas ---
# La fórmula es (x - min) / (max - min)
# Esto se hace de forma eficiente para todas las columnas de una vez.
for col in cols_to_scale:
    min_val = min_max_values.loc[col, 'min']
    max_val = min_max_values.loc[col, 'max']
    # Evitar división por cero si max == min (lo que significa que todos los valores son iguales)
    if (max_val - min_val) != 0:
        data[col] = (data[col] - min_val) / (max_val - min_val)
    else:
        # Si todos los valores son iguales, la escala min-max los convierte en 0 o 0.5 (convención)
        # Aquí los mantenemos en 0 si es el caso, o 0.5 si quieres un punto medio
        data[col] = 0.0 # O np.nan si prefieres marcarlo como indefinido para la escala.

print("\nMin-Max Scaling aplicado a las variables numéricas (excepto coordenadas).")

# --- 4. Mostrar un resumen de los datos escalados ---
print("\nEstadísticas del DataFrame escalado (primeras 5 filas):")
print(data.head())
print("\nEstadísticas del DataFrame escalado (summary):")
print(data.describe())


Min-Max Scaling aplicado a las variables numéricas (excepto coordenadas).

Estadísticas del DataFrame escalado (primeras 5 filas):
         POINT_X       POINT_Y  PLA         A        AC       CIB     CICCB  \
0  722396.507981  4.365860e+06  0.0  0.604254  0.067778  0.203910  0.390004   
1  722404.245816  4.365860e+06  0.0  0.581583  1.000000  0.201362  0.388246   
2  722411.983652  4.365860e+06  0.0  0.581583  1.000000  0.198815  0.386458   
3  722396.502336  4.365868e+06  0.0  0.609629  1.000000  0.204328  0.391055   
4  722404.240164  4.365868e+06  0.0  0.608561  1.000000  0.201762  0.389291   

  CUS       DBB       DOB OC         P  
0   5  0.620621  0.000000  4  0.077332  
1   1  0.614835  0.788748  4  0.089864  
2   1  0.610494  0.783177  4  0.089864  
3   1  0.620342  0.000000  4  0.051674  
4   1  0.614553  0.788748  4  0.062030  

Estadísticas del DataFrame escalado (summary):
             POINT_X       POINT_Y           PLA             A            AC  \
count   64345.00000